In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

plt.style.use("ggplot")

In [ ]:
ticker_symbol = "TSLA"
prices = yf.download(ticker_symbol, period="2y")
prices.columns = ["_".join([c for c in col if c])for col in prices.columns]
prices.head()

In [ ]:
# Calculation of daily percentage return

prices["Daily_Return"] = prices["Close_TSLA"].pct_change()

# 20 day Moving Average
prices["SMA20"] = prices["Close_TSLA"].rolling(window=20).mean()

prices.tail()

In [ ]:
# Plotting the price and SMA20

prices[["Close_TSLA", "SMA20"]].plot(figsize=(12,5))
plt.title(f"{ticker_symbol} Price vs 20-day SMA")
plt.ylabel("Price ($)")
plt.show()

In [ ]:
prices["Signal"] = (prices["Close_TSLA"] > prices["SMA20"]).astype(int)

# flattening the multiindex columns

In [ ]:
prices["Strategy_Return"] = prices["Daily_Return"] * prices["Signal"].shift(1).fillna(0)
buy_hold = (1+prices["Daily_Return"].fillna(0)).cumprod()
strategy = (1+prices["Strategy_Return"].fillna(0)).cumprod()

results = pd.DataFrame({
    "Buy_and_Hold": buy_hold,
    "Strategy": strategy
})

results.tail()

In [ ]:
results.plot(figsize=(12,6))
plt.title(f"{ticker_symbol} Beginner Strategy vs Buy-and-Hold")
plt.ylabel("Growth of $1")
plt.xlabel("Date")
plt.show()

In [ ]:
tot_trades = prices["Signal"].diff().abs().sum()

final_buy_hold = buy_hold.iloc[-1]
final_strat = strategy.iloc[-1]

print(f"Total trades by strategy = {int(tot_trades)}")
print(f"Final Buy-Hold growth of $1 = {final_buy_hold:.2f}")
print(f"Final Strategy growth of $1 = {final_strat:.2f}")

if final_strat > final_buy_hold:
    print("Strategy worked pretty well!")
else:
    print("Strategy didn't work too great pal")

In [ ]:
# Troubleshootin

print(prices.columns)
all(isinstance(col, tuple) for col in prices.columns) # was true with multiindex columns, now false with flattened columns. talk about this in the readme


In [ ]:
prices.tail()
